In [ ]:
import numpy as np
import pandas as pd
import re
import io
import requests
import holoviews as hv
import hvplot.pandas
import panel as pn
import pint
from requests_html import HTMLSession

In [ ]:
%load_ext autoreload
%autoreload 2
import util

In [ ]:
hv.extension("bokeh")

# Spectral response

In [ ]:
def read_spectrum(id_, bins=None):
    df = pd.read_csv(
        f"https://www.fpbase.org/spectra_csv/?q={id_}", index_col="wavelength"
    ).squeeze()
    if bins is not None:
        df = util.interpolate_dataframe(df, bins)
    return df


def get_fpbase_spectra(bins=None):
    fpbase_entries = requests.get("https://www.fpbase.org/api/proteins/spectra/").json()
    fps = {}
    for fpbase_entry in fpbase_entries:
        fp = {}
        spectra = []
        for spectrum_type in fpbase_entry["spectra"]:
            state_name = re.sub(r"^default_", "", spectrum_type["state"])
            for k, v in spectrum_type.items():
                if k == "state":
                    continue
                elif k == "data":
                    spectra.append(
                        pd.DataFrame(v, columns=["wavelength", state_name]).set_index(
                            "wavelength"
                        )
                    )
                else:
                    fp[f"{state_name}_{k}"] = v
        df = pd.concat(spectra, axis=1)
        if bins is not None:
            df = util.interpolate_dataframe(df, bins)
        fp["spectra"] = df
        fps[fpbase_entry["name"]] = fp
    return fps

In [ ]:
bins = np.arange(300, 1000)

In [ ]:
%%time
sources = {"SOLIS-3C": "7016", "SOLIS-565C": "7004"}

dichroics = {
    "Chroma ZT514rdc": "560",
    "Chroma ZT532rdc": "645",
    "Chroma T550lpxr": "803",
    "Chroma T556lpxr": "658",
    "Chroma ZT561rdc": "523",
    "Chroma ZT561rdc-xr": "708",
    "Chroma ZT568rdc": "604",
    "Chroma T570lpxr": "427",
    "Chroma T590lpxr": "593",
    "Chroma ZT594rdc": "569",
    "Chroma T600lpxr": "609",
    "Chroma T610lpxr": "601",
    "Chroma ZT633rdc": "668",
    "Chroma T635lpxr": "616",
    "Chroma ZT640rdc": "439",
}

longpass_filters = {
    "Chroma ET520LP": "760",
    "Chroma ET525lp": "530",
    "Chroma ET542lp": "461",
    "Chroma ET570lp": "487",
    "Chroma ET575lp": "620",
    "Chroma ET590lp": "805",
    "Chroma ET610lp": "350",
    "Chroma RET638lp": "2855",
    "Chroma ET655lp": "683",
    "Chroma ET665lp": "573",
}

sources = {name: read_spectrum(q, bins=bins) for name, q in sources.items()}
dichroics = {name: read_spectrum(q, bins=bins) for name, q in dichroics.items()}
longpass_filters = {
    name: read_spectrum(q, bins=bins) for name, q in longpass_filters.items()
}

In [ ]:
def get_semrock_filters(urls, bins=None):
    if isinstance(urls, str):
        urls = [urls]
    session = HTMLSession()
    spectra = {}
    for url in urls:
        html = session.get(url).html
        catalog_numbers = [
            a.text for a in html.find("#resultsView .cartSection > h1 > a")
        ]
        for catalog_number in catalog_numbers:
            spectrum_number = "-".join(catalog_number.split("-")[:2]).replace("/", "_")
            name = f"Semrock {spectrum_number}"
            spectrum_urls = [
                f"https://www.semrock.com/_ProductData/Spectra/{spectrum_number}_Spectrum.txt",
                f"https://www.semrock.com/_ProductData/Spectra/{spectrum_number}_DesignSpectrum.txt",
            ]
            spectrum = None
            for spectrum_url in spectrum_urls:
                res = session.get(spectrum_url)
                if not res.ok:
                    continue
                lines = io.StringIO(
                    "\n".join(
                        [l for l in res.text.split("\n") if not l.startswith("---")]
                    )
                )
                spectrum = pd.read_csv(
                    lines,
                    sep="\t",
                    skiprows=4,
                    names=["wavelength", spectrum_number],
                    index_col=0,
                ).squeeze()
                break
            if spectrum is None:
                raise ValueError(f"could not find spectrum for '{spectrum_number}'")
            if bins is not None:
                spectrum = util.interpolate_dataframe(spectrum, bins)
            spectra[name] = spectrum
    return spectra

In [ ]:
%%time
semrock_dichroic_urls = [
    "https://www.semrock.com/filtersRefined.aspx?minWL=520&maxWL=660&id=497,800&page=1&so=0&recs=10000"
]
semrock_longpass_urls = [
    f"https://www.semrock.com/filtersRefined.aspx?minWL=520&maxWL=660&id=21,{id_}&page=1&so=0&recs=10000"
    for id_ in (537, 538, 545)
]

dichroics = {**dichroics, **get_semrock_filters(semrock_dichroic_urls, bins=bins)}
longpass_filters = {
    **longpass_filters,
    **get_semrock_filters(semrock_longpass_urls, bins=bins),
}

In [ ]:
%%time
fps = get_fpbase_spectra(bins=bins)

In [ ]:
def _filter_plot(fp_name, dc_name, lp_name):
    return (
        fps[fp_name]["spectra"].fillna(0).hvplot()
        * dichroics[dc_name].hvplot()
        * longpass_filters[lp_name].hvplot()
    )


def filter_plot(
    fp_names=fps.keys(), dc_names=dichroics.keys(), lp_names=longpass_filters.keys()
):
    layout = pn.interact(
        _filter_plot, fp_name=fp_names, dc_name=dc_names, lp_name=lp_names
    )
    return pn.Row(pn.Column("## Filters", layout[0]), layout[1])

## Filters

In [ ]:
def combine_filter(dc, lp, threshold=0.9, max_wavelength=700):
    dc = dc.loc[:max_wavelength]
    lp = lp.loc[:max_wavelength]
    log_dc = np.log10(dc)
    log_lp = np.log10(lp)
    # highest wavelength that gives >90% excitation (dichroic reflectance)
    ex_cutoff = (dc > 1 - threshold)[::-1].idxmin()
    # highest wavelength that gives <90% transmission (dichroic+lp)
    em_cutoff = (log_dc + log_lp > np.log10(threshold)).idxmax()
    # gap between those wavelengths
    gap = em_cutoff - ex_cutoff
    # highest OD below excitation cutoff
    worst_rejection = (log_dc + log_lp).loc[:ex_cutoff].max()
    return pd.Series(
        {
            "ex_cutoff": ex_cutoff,
            "em_cutoff": em_cutoff,
            "gap": gap,
            "worst_rejection": worst_rejection,
        }
    )


filter_combinations = pd.concat(
    {
        dc_name: pd.concat(
            {
                lp_name: combine_filter(dc_spectrum, lp_spectrum)
                for lp_name, lp_spectrum in longpass_filters.items()
            },
            axis=1,
        ).T
        for dc_name, dc_spectrum in dichroics.items()
    }
).sort_values("ex_cutoff")

In [ ]:
acceptable_filter_combinations = filter_combinations[
    (filter_combinations["gap"] < 30) & (filter_combinations["worst_rejection"] < -5.5)
]

In [ ]:
# for each dichroic, pick the longpass that minimizes the gap
selected_filter_combinations = (
    acceptable_filter_combinations.groupby(level=0)
    .apply(lambda x: x.loc[[x["gap"].idxmin()]])
    .droplevel(0)
    .sort_values("ex_cutoff")
    .index.values
)

## FPs

In [ ]:
longpass_filters.keys()

In [ ]:
filter_combinations.loc["Semrock Di03-R635"]

In [ ]:
filter_combinations[
    (filter_combinations["gap"] < 50) & (filter_combinations["worst_rejection"] < -5)
]

In [ ]:
acceptable_filter_combinations

In [ ]:
selected_filter_combinations

In [ ]:
def show_heatmap(df, highlight_negative=False, **kwargs):
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        df = df.style.format(precision=2).background_gradient(
            **{"cmap": "RdPu", "axis": None, **kwargs}
        )
        if highlight_negative:

            def style_negative(v, props=""):
                return props if v < 0 else None

            df = df.applymap(
                style_negative,
                props=f"background-color:{highlight_negative};color:white;",
            )
        display(df)


def fp_efficiency(dc, lp, ex, em, ex_cutoff, em_cutoff):
    return pd.Series(
        {
            ("efficiency", "ex"): ((1 - dc) * (ex / ex.sum())).sum(),
            ("efficiency", "em"): (dc * lp * (em / em.sum())).sum(),
            ("cutoff_amplitude", "ex"): ex.loc[ex_cutoff],
            ("cutoff_amplitude", "em"): em.loc[em_cutoff],
            ("margin", "ex"): ex_cutoff - ex.idxmax(),
            ("margin", "em"): em.idxmax() - em_cutoff,
        }
    )


def evaluate_filter_combinations(
    filter_combo_names, fp_names, dichroics, longpass_filters, filter_combinations, fps
):
    d = {}
    for filter_combo in filter_combo_names:
        dc = dichroics[filter_combo[0]]
        lp = longpass_filters[filter_combo[1]]
        dd = {}
        for fp_name in fp_names:
            ex = fps[fp_name]["spectra"]["ex"]
            em = fps[fp_name]["spectra"]["em"]
            dd[fp_name] = (
                fp_efficiency(
                    dc,
                    lp,
                    ex,
                    em,
                    filter_combinations.loc[filter_combo]["ex_cutoff"],
                    filter_combinations.loc[filter_combo]["em_cutoff"],
                )
                .to_frame()
                .T
            )
        d[filter_combo] = (
            pd.concat(dd, axis=1)
            .reorder_levels([1, 2, 0], axis=1)
            .sort_index(axis=1, level=1, sort_remaining=False)
        )
    return pd.concat(d, axis=0).droplevel(-1)

In [ ]:
selected_fps = [
    "mScarlet-I",
    "mScarlet-H",
    "mCherry",
    "mCherry2",
    "mKate2",
    "E2-Crimson",
    "TurboRFP",
]
selected_fps = sorted(selected_fps, key=lambda x: fps[x]["ex_max"])
filter_metrics = evaluate_filter_combinations(
    selected_filter_combinations,
    selected_fps,
    dichroics,
    longpass_filters,
    filter_combinations,
    fps,
)

In [ ]:
show_heatmap(filter_metrics["efficiency"])

In [ ]:
show_heatmap(filter_metrics["cutoff_amplitude"])

In [ ]:
show_heatmap(filter_metrics["margin"], vmin=0, highlight_negative="black")

In [ ]:
filter_plot(fp_names=selected_fps)

In [ ]:
filter_plot(fp_names=selected_fps)

In [ ]:
filter_plot(fp_names=selected_fps)

In [ ]:
filter_plot(fp_names=selected_fps)

In [ ]:
filter_plot(fp_names=selected_fps)

In [ ]:
", ".join(fps.keys())